<a href="https://colab.research.google.com/github/atlas-github/nus_cloudcomputing/blob/main/Google_News_RSS_Feed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip install feedparser pandas google-cloud-bigquery

In [ ]:
import feedparser
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# RSS feed URL
rss_url = "https://news.google.com/rss/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx1YlY4U0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl=US&ceid=US:en"

# BigQuery setup
project_id = "### INSERT PROJECT ID ###"
dataset_id = "### INSERT DATASET ID ###"
table_id = "### INSERT TABLE ID ###"
credentials = service_account.Credentials.from_service_account_file("### INSERT CREDENTIAL FILE NAME ###")
client = bigquery.Client(credentials=credentials, project=project_id)

# Email setup
smtp_server = "smtp.gmail.com"
smtp_port = 587
email_sender = "### INSERT SENDER EMAIL ###"
email_password = "### INSERT APP PASSWORD ###"
email_recipient = "### INSERT RECIPIENT EMAIL ###"

# Parse the RSS feed
feed = feedparser.parse(rss_url)
entries = []
for entry in feed.entries:
    entries.append({
        "Title": entry.title,
        "Link": entry.link,
        "Published": entry.published,
        "Summary": entry.summary if "summary" in entry else ""
    })

# Create a DataFrame
df = pd.DataFrame(entries)

# Load the existing data from BigQuery (if any)
query = f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`"
try:
    existing_data = client.query(query).to_dataframe()
except Exception as e:
    existing_data = pd.DataFrame(columns=["Title", "Link", "Published", "Summary"])

# Find new rows
new_rows = df[~df["Link"].isin(existing_data["Link"])]

# Insert new rows into BigQuery
if not new_rows.empty:
    job = client.load_table_from_dataframe(new_rows, f"{project_id}.{dataset_id}.{table_id}")
    job.result()  # Wait for the job to complete

    # Send email notification for new rows
    html_table = new_rows.to_html(index=False, escape=False)
    email_subject = "New RSS Feed Entries"
    email_body = f"""
    <html>
    <body>
        <p>New entries have been added to the RSS feed:</p>
        {html_table}
    </body>
    </html>
    """

    # Create the email message
    message = MIMEMultipart()
    message["From"] = email_sender
    message["To"] = email_recipient
    message["Subject"] = email_subject
    message.attach(MIMEText(email_body, "html"))

    # Send the email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(email_sender, email_password)
        server.sendmail(email_sender, email_recipient, message.as_string())

    print("New rows inserted into BigQuery and email notification sent.")
else:
    print("No new rows to insert.")
